1\. Write a function that converts number representation (bin<->dec<->hex)

In [6]:
#Exercise 1)
def conversion(a):
    y=hex(a)
    z=bin(a)
    x=int(a)
    return y,z,x
a,b,c= conversion(16)
print(a)
print(b)
print(c)

0x10
0b10000
16


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [14]:
def conv(word):
    mant=1
    for n in range(1,23):
        mant=mant + int(word[8+n])/(2**n)
    return (-1)**int(word[0])*mant*2**(int(word[1:9],2)-127)
word="10111110011000000000000000000000"
conv(word)

-0.21875

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [2]:
#Exercise 3) 

a=1.
b=1
while b!=0:
    c=b
    b=b/2
print("underflow =",c)

while a!= float("inf") :
    d=a
    a=a*2.
print("overflow =",d)
    

underflow = 5e-324
overflow = 8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [4]:
#Exercise 4
a=1
c=0
while (2-c)!=0.:
    b=c
    c=c+a
    a=a/2
    
print("precision =",2-b)

precision = 2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [5]:
#Exercise 5

#a)
import numpy as np
def fquad(a,b,c):
    x=(-b+(b*b-4*a*c)**0.5)/(2*a)
    y=(-b-(b*b-4*a*c)**0.5)/(2*a)
    return x,y
       
x, y = fquad(0.001,1000,0.001)
print(x,y)

#b)
def fquad2(a,b,c):
    x=2*c/(-b-(b**2-4*a*c)**0.5)
    y=2*c/(-b+(b**2-4*a*c)**0.5)
    return x,y
x, y = fquad2(0.001,1000,0.001)
print(x,y)


#The smaller result of a) is more accurate that the smaller result of b)
#The smaller result of b) is more accurate that the smaller result of a)
#

#c)


-9.999894245993346e-07 -999999.999999
-1.000000000001e-06 -1000010.5755125057


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [11]:
#Exercise 6
def fun(x):
    return x*(x-1)


x=1
for i in range(1,8):
    delta=10**(-2*i)
    y=(fun(x+delta)-fun(x))/delta
    print(y)
    
#We have the best accurancy for delta=10^(-8)

1.010000000000001
1.0000999999998899
1.0000009999177333
1.0000000039225287
1.000000082840371
1.0000889005833413
0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [10]:
#Exercise 7
y=0
N=150000000
dk=2/N
k=-1
import time
t0=time.perf_counter()
for i in range(1,N+1):
    y=y+(1-(k+dk*i)**2)**0.5*dk
print(y)
t1=time.perf_counter()
print(t1-t0)

#running the program for 1 second means increas N to 2400000, and the error it's on the 10th decimal
#running the program for 1 minute means increas N to 150000000, and there is no visible error
#

1.5707963267944103
63.28707773200017
